In [8]:
from  langchain_openai import ChatOpenAI
import os 
from dotenv import load_dotenv

# This function will load all the variables from the .env file and will 
# make them available in the os.environ dictionary (env variables)
load_dotenv() 

if os.environ.get("OPENAI_API_KEY"):
    print("Bro API KEY Variable exists")
else:
    raise ValueError("OPENAI_API_KEY not found")

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from  langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser

llm_openai = ChatOpenAI(model="gpt-5-mini",temperature=0)

Bro API KEY Variable exists


In [16]:
from pydantic import BaseModel
from typing import Literal

class llm_schema(BaseModel):
    movie_summary_flag: Literal["positive", "negative"]

llm_structured_output = llm_openai.with_structured_output(llm_schema)


# **CHAIN WITH Conditional Chains**

In [18]:
# TASK -1 [Prompt]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie review evaluator"),
    ("human", "Please categorize the movie review as positive or negative : {input}")])

In [17]:
# TASK - 2 [LLM]

llm_structured_output = llm_openai.with_structured_output(llm_schema)


In [22]:
# TASK - 3 [Custom Runnable]
from langchain_core.runnables import RunnableLambda

def pydantic_json(input:llm_schema)-> str:

    return input.model_dump()['movie_summary_flag']

pydantic_json_lambda = RunnableLambda(pydantic_json)


### **Conditional Chain 1**

In [23]:
# TASK - 1 [Prompt]

linkedin_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a LinkedIn post generator"),
    ("human", "Create a post for the following text for LinkedIn: {text}")])

# TASK - 2 [LLM]

llm_openai = ChatOpenAI(model="gpt-5-mini",temperature=0)

# TASK - 3 [Str Parser]

str_parser = StrOutputParser()

chain_linkedin = linkedin_prompt | llm_openai | str_parser

### **Conditional Chain 2**

In [26]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnableBranch

In [27]:
def insta_chain(text:dict):

    text = text["text"]

    # TASK - 1 [Prompt]
    insta_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a LinkedIn post generator"),
    ("human", "Create a post for the following text for Instagram: {text}")])
    
    # TASK - 2 [LLM]
    llm_openai = ChatOpenAI(model="gpt-5-mini",temperature=0)

    # TASK - 3 [Str Parser]
    str_parser = StrOutputParser()

    chain_insta = insta_prompt | llm_openai | str_parser

    result = chain_insta.invoke(text)

    return result

insta_chain_runnable = RunnableLambda(insta_chain)


### **Final Orchestration**

In [31]:
conditional_chain = RunnableBranch(
    (lambda x: "positive" in x, chain_linkedin),
     insta_chain_runnable
)

final_orchestrator = prompt_template | llm_structured_output | pydantic_json_lambda | conditional_chain

In [32]:
final_orchestrator.invoke({"input": "I loved this KGF movie"})

"Positivity isn't just a mood — it's a strategy. When we choose to focus on solutions, acknowledge small wins, and bring curiosity instead of judgment to setbacks, teams become more resilient and productive. \n\nToday I challenged myself to start meetings by sharing one recent win, however small. The shift in tone was immediate: people came in more engaged, ideas flowed more freely, and follow-ups actually happened. \n\nSmall consistent choices shape culture. What one positive habit could you start this week?\n\n#positivity #mindset #leadership #growth #wellbeing"